# 19_OpenAPI 기반의 지도 시각화

In [15]:
%%html
<h1>
위도 : <span id="lat"></span>, 경도 : <span id="lng"></span>
</h1>

<script>
navigator.geolocation.getCurrentPosition(function(position){
    document.getElementById("lat").innerHTML = position.coords.latitude;
    document.getElementById("lng").innerHTML = position.coords.longitude;
});
</script>

## #01. 기본 준비 단계

### 1) 필요한 패키지 가져오기 

In [16]:
import folium
import requests
import json
import pandas as pd
from pandas import DataFrame
import numpy

### 2) 위도, 경도 변수 준비하기 

In [17]:
lat = 37.4923615
lng = 127.02928809999999
m = 5000    #반경 5km

### 3) 세션 생성

In [18]:
user_agent = "Mozilla/5.0 (Linux; Android 5.0; SM-G900P Build/LRX21T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Mobile Safari/537.36"
session = requests.Session()
session.headers.update({'User-agent':user_agent, 'referer':None})

## #02. 특정 위치 부근의 공정 마스크 판매 정보

공공 데이터 포털에서 공적 마스크 판매 정보와 관련된 OpenAPI를 제공한다.

> https://www.data.go.kr/dataset/15043025/openapi.do <br>
> https://app.swaggerhub.com/apis-docs/Promptech/public-mask-info/20200307-oas3#/

### 1) 접속 주소 준비하기 

#### 변수가 치환될 주소 템플릿

In [19]:
url_tpl = "https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/storesByGeo/json?lat={lat}&lng={lng}&m={m}"

#### 최종 주소 확인하기 

In [20]:
url = url_tpl.format(lat=lat, lng=lng, m=m)
url

'https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/storesByGeo/json?lat=37.4923615&lng=127.02928809999999&m=5000'

### 2) API를 통한 JSON 테이터 가져오기 

#### OpenAPI를 통한 JSON 가져오기

In [21]:
r = session.get(url)
r.text

'{\n  "count": 723,\n  "stores": [{\n    "addr": "서울특별시 관악구 남부순환로 2054 1층 (남현동)",\n    "code": "12868957",\n    "created_at": "2020/03/28 09:40:00",\n    "lat": 37.476344,\n    "lng": 126.9769436,\n    "name": "워너비사랑약국",\n    "remain_stat": "few",\n    "stock_at": "2020/03/27 12:18:00",\n    "type": "01"\n  }, {\n    "addr": "서울특별시 관악구 남부순환로 2056 태림빌딩 지하1층 (남현동)",\n    "code": "12869538",\n    "created_at": "2020/03/28 09:40:00",\n    "lat": 37.4763242,\n    "lng": 126.9772094,\n    "name": "아이봄약국",\n    "remain_stat": "break",\n    "stock_at": "2020/03/27 09:56:00",\n    "type": "01"\n  }, {\n    "addr": "서울특별시 동작구 남부순환로 2055 (사당동)",\n    "code": "12830925",\n    "created_at": "2020/03/28 09:40:00",\n    "lat": 37.4768791,\n    "lng": 126.9771081,\n    "name": "팜프라자약국",\n    "remain_stat": "few",\n    "stock_at": "2020/03/27 13:02:00",\n    "type": "01"\n  }, {\n    "addr": "서울특별시 관악구 남현길 7 한일프라자 1층 (남현동)",\n    "code": "12869228",\n    "created_at": "2020/03/28 09:45:00",\n    "lat":

#### JSON을 딕셔너리로 변환

In [22]:
mask_dict = json.loads(r.text)
mask_dict

{'count': 723,
 'stores': [{'addr': '서울특별시 관악구 남부순환로 2054 1층 (남현동)',
   'code': '12868957',
   'created_at': '2020/03/28 09:40:00',
   'lat': 37.476344,
   'lng': 126.9769436,
   'name': '워너비사랑약국',
   'remain_stat': 'few',
   'stock_at': '2020/03/27 12:18:00',
   'type': '01'},
  {'addr': '서울특별시 관악구 남부순환로 2056 태림빌딩 지하1층 (남현동)',
   'code': '12869538',
   'created_at': '2020/03/28 09:40:00',
   'lat': 37.4763242,
   'lng': 126.9772094,
   'name': '아이봄약국',
   'remain_stat': 'break',
   'stock_at': '2020/03/27 09:56:00',
   'type': '01'},
  {'addr': '서울특별시 동작구 남부순환로 2055 (사당동)',
   'code': '12830925',
   'created_at': '2020/03/28 09:40:00',
   'lat': 37.4768791,
   'lng': 126.9771081,
   'name': '팜프라자약국',
   'remain_stat': 'few',
   'stock_at': '2020/03/27 13:02:00',
   'type': '01'},
  {'addr': '서울특별시 관악구 남현길 7 한일프라자 1층 (남현동)',
   'code': '12869228',
   'created_at': '2020/03/28 09:45:00',
   'lat': 37.4760703,
   'lng': 126.9776837,
   'name': '마음약국',
   'remain_stat': 'empty',
   'stock

#### 딕셔너리를 데이터프레임으로 변환

In [23]:
mask_df = DataFrame(mask_dict['stores'])
mask_df

,addr,code,created_at,lat,lng,name,remain_stat,stock_at,type
0,서울특별시 관악구 남부순환로 2054 1층 (남현동),12868957,2020/03/28 09:40:00,37.476344,126.976944,워너비사랑약국,few,2020/03/27 12:18:00,01
1,서울특별시 관악구 남부순환로 2056 태림빌딩 지하1층 (남현동),12869538,2020/03/28 09:40:00,37.476324,126.977209,아이봄약국,break,2020/03/27 09:56:00,01
2,서울특별시 동작구 남부순환로 2055 (사당동),12830925,2020/03/28 09:40:00,37.476879,126.977108,팜프라자약국,few,2020/03/27 13:02:00,01
3,서울특별시 관악구 남현길 7 한일프라자 1층 (남현동),12869228,2020/03/28 09:45:00,37.476070,126.977684,마음약국,empty,2020/03/27 10:03:00,01
4,서울특별시 관악구 남부순환로 2062 (남현동),11810629,2020/03/28 09:40:00,37.476398,126.977794,신한성약국,empty,2020/03/27 11:54:00,01
...,...,...,...,...,...,...,...,...,...
718,"서울특별시 강남구 영동대로 722 (청담동, 풍양빌딩)",12824691,2020/03/28 09:40:00,37.522385,127.056412,청담제일약국,break,2020/03/28 09:12:00,01
719,"서울특별시 강남구 학동로101길 26 102호 (청담동, 삼익상가)",11800721,2020/03/28 09:40:00,37.522487,127.057706,새휘정약국,some,2020/03/27 18:53:00,01
720,"서울특별시 서초구 중앙로 582 103호 (우면동, 서초타워)",11887982,2020/03/28 09:40:00,37.457149,127.015332,태평양약국,few,2020/03/28 08:47:00,01
721,"서울특별시 서초구 헌릉로8길 45 2층 203호 (내곡동, 서초 포레스타 2단지)",12867110,2020/03/28 09:40:00,37.455551,127.060950,봄약국,few,2020/03/28 08:42:00,01


## #03. 지도 시각화

### 1) 지도 객체 생성

#### 지도의 중심이 되는 위도와 경도를 설정

In [24]:
# zoom_start : 배열 1-22
map_osm = folium.Map(location=[lat, lng], zoom_start=17)
map_osm

In [25]:
# 마스크 입고 상태 (재고 상태 )
conditions = [  (mask_df['remain_stat'] == 'plenty'), # 100개 이상
                (mask_df['remain_stat'] == 'some'),   # 30개이상 100개 미만
                (mask_df['remain_stat'] == 'few'),    # 2개이상 30개 미만
                (mask_df['remain_stat'] == 'empty'),  # 1개 이하
                (mask_df['remain_stat'] == 'break')]  # 판매중지

# 조건에 따라 부여될 학점
states = [ '100개 이상', '30개 이상 100개미만', '2개이상 30개 미만', '1개 이하', '판매중지']
colors = [ 'green', 'orange', 'red', 'gray', 'black']

# 조건에 따른 학점열 추가하기
mask_df['states'] = numpy.select(conditions, states)
mask_df['colors'] = numpy.select(conditions, colors)

mask_df

,addr,code,created_at,lat,lng,name,remain_stat,stock_at,type,states,colors
0,서울특별시 관악구 남부순환로 2054 1층 (남현동),12868957,2020/03/28 09:40:00,37.476344,126.976944,워너비사랑약국,few,2020/03/27 12:18:00,01,2개이상 30개 미만,red
1,서울특별시 관악구 남부순환로 2056 태림빌딩 지하1층 (남현동),12869538,2020/03/28 09:40:00,37.476324,126.977209,아이봄약국,break,2020/03/27 09:56:00,01,판매중지,black
2,서울특별시 동작구 남부순환로 2055 (사당동),12830925,2020/03/28 09:40:00,37.476879,126.977108,팜프라자약국,few,2020/03/27 13:02:00,01,2개이상 30개 미만,red
3,서울특별시 관악구 남현길 7 한일프라자 1층 (남현동),12869228,2020/03/28 09:45:00,37.476070,126.977684,마음약국,empty,2020/03/27 10:03:00,01,1개 이하,gray
4,서울특별시 관악구 남부순환로 2062 (남현동),11810629,2020/03/28 09:40:00,37.476398,126.977794,신한성약국,empty,2020/03/27 11:54:00,01,1개 이하,gray
...,...,...,...,...,...,...,...,...,...,...,...
718,"서울특별시 강남구 영동대로 722 (청담동, 풍양빌딩)",12824691,2020/03/28 09:40:00,37.522385,127.056412,청담제일약국,break,2020/03/28 09:12:00,01,판매중지,black
719,"서울특별시 강남구 학동로101길 26 102호 (청담동, 삼익상가)",11800721,2020/03/28 09:40:00,37.522487,127.057706,새휘정약국,some,2020/03/27 18:53:00,01,30개 이상 100개미만,orange
720,"서울특별시 서초구 중앙로 582 103호 (우면동, 서초타워)",11887982,2020/03/28 09:40:00,37.457149,127.015332,태평양약국,few,2020/03/28 08:47:00,01,2개이상 30개 미만,red
721,"서울특별시 서초구 헌릉로8길 45 2층 203호 (내곡동, 서초 포레스타 2단지)",12867110,2020/03/28 09:40:00,37.455551,127.060950,봄약국,few,2020/03/28 08:42:00,01,2개이상 30개 미만,red


## 03. 지도 시각화

In [26]:
# zoom_start : 배열 1-22
map_osm = folium.Map(location=[lat, lng], zoom_start=17)
map_osm

### 2)지도객체에 마커 추가

데이터프레임의 행 수 만큼 반복하면서 지정

In [28]:
for i in mask_df.index:
    x = mask_df.loc[i, 'lat']
    y = mask_df.loc[i, 'lng']
    name = mask_df.loc[i, 'name']
    state = mask_df.loc[i, 'states']
    color = mask_df.loc[i, 'colors']
    print(x, y, name, state, color)
    
    # HTML을 사용한 팝업
    tag_tpl = "<div style='white-space: nowrap'><b>%s</b><br>%s</div>"
    tag = tag_tpl % (name, state)
    popup_html = folium.Popup(tag, parse_html=False)

    # 마커 객체 생성
    mk = folium.Marker([x, y], popup=popup_html, icon=folium.Icon(color=color))

    mk.add_to(map_osm)  # 마커 객체를 지도에 추가함
    

37.476344 126.9769436 워너비사랑약국 2개이상 30개 미만 red
37.4763242 126.9772094 아이봄약국 판매중지 black
37.4768791 126.9771081 팜프라자약국 2개이상 30개 미만 red
37.4760703 126.9776837 마음약국 1개 이하 gray
37.4763977 126.9777939 신한성약국 1개 이하 gray
37.4769407 126.9773807 파란약국 100개 이상 green
37.4769192 126.9775136 평화약국 1개 이하 gray
37.4777602 126.9781039 온누리세종약국 1개 이하 gray
37.4715201 126.9824756 사랑찬약국 1개 이하 gray
37.4743938 126.9814601 사당미소약국 30개 이상 100개미만 orange
37.476161 126.9803937 사랑약국 2개이상 30개 미만 red
37.4759484 126.9811803 사당365약국 1개 이하 gray
37.4771747 126.9811278 사당소화약국 판매중지 black
37.4770623 126.9822612 사당13번약국 1개 이하 gray
37.47875 126.9813575 새미약국 30개 이상 100개미만 orange
37.4763378 126.9865321 메디팜메트로약국 30개 이상 100개미만 orange
37.4768774 126.9867478 강남사랑약국 2개이상 30개 미만 red
37.4762829 126.9875282 나은약국 30개 이상 100개미만 orange
37.4797056 126.9984853 경희정산한약국 판매중지 black
37.4778415 127.0003899 비타민약국 30개 이상 100개미만 orange
37.4781622 126.999425 주앙약국 30개 이상 100개미만 orange
37.4781182 127.0001766 한영온누리약국 30개 이상 100개미만 orange
37.4830336 126.97488

c:\users\ezen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: UserWarning: color argument of Icon should be one of: {'black', 'lightred', 'blue', 'lightgreen', 'darkblue', 'gray', 'cadetblue', 'beige', 'white', 'green', 'orange', 'darkred', 'pink', 'lightblue', 'red', 'darkpurple', 'lightgray', 'purple', 'darkgreen'}.
  from ipykernel import kernelapp as app


37.5169298 127.0223189 오렌지약국 1개 이하 gray
37.5011826 127.0255738 새드림약국 판매중지 black
37.5013657 127.0255201 스타약국 판매중지 black
37.5015461 127.0254021 아임약국 2개이상 30개 미만 red
37.501235 127.0266249 강남웰약국 1개 이하 gray
37.5020128 127.0244608 아이약국 1개 이하 gray
37.5023687 127.0248894 새열린약국 1개 이하 gray
37.502832 127.0245612 메디안약국 0 0
37.5024003 127.0259642 행복한수약국 1개 이하 gray
37.5034826 127.024458 강남약국 100개 이상 green
37.5038061 127.0241492 가온약국 2개이상 30개 미만 red
37.5041598 127.0251636 더블유약국 100개 이상 green
37.5043479 127.0250552 레이첼약국 100개 이상 green
37.5049158 127.0248029 신논현비비약국 100개 이상 green
37.5049634 127.0250571 고은약국 판매중지 black
37.5051136 127.0246963 제일그랜드약국 1개 이하 gray
37.5045463 127.0256122 영원약국 100개 이상 green
37.5050767 127.0255834 신논현약국 100개 이상 green
37.5051313 127.0253462 웰스약국 판매중지 black
37.5053198 127.0264873 별하약국 100개 이상 green
37.5010881 127.0342169 경희자연한약국 판매중지 black
37.504318 127.0341902 역삼라이프약국 판매중지 black
37.50591 127.0242904 건강한세상행복한약국 판매중지 black
37.5062114 127.0241192 블루밍약국 판매중지 black
37.507009 127.026

In [30]:
map_osm